In [2]:
import os
import networkx as nx
from tqdm import tqdm

In [3]:
os.getcwd()

'/mnt/c/Users/tduri/Development/workspace/GraphRec-WWW19-myfork/GraphRec-WWW19/notebooks'

In [18]:
def create_ratings_map(filename):
    ratings = set()
    ratings_map = {}
    id_counter = 0

    with open(filename) as fr:
        for line in fr.readlines():
            tokens = line.split()
            rating = float(tokens[2])
            ratings.add(rating)

    ratings = list(ratings)
    ratings.sort()

    for rating in ratings:
        if rating not in ratings_map:
            ratings_map[rating] = id_counter
            id_counter += 1

    return ratings_map

def get_users_and_items_from_ratings_file(filename, ratings_map):
    users = set()
    items = set()
    user_items_dict = {}
    item_users_dict = {}
    user_item_ratings = {}
    with open(filename) as fr:
        for line in fr.readlines():
            tokens = line.split()
            user_id = int(tokens[0])
            item_id = int(tokens[1])
            rating = ratings_map[float(tokens[2])]
            if len(tokens) == 4:
                timestamp = int(tokens[3])

            users.add(user_id)
            items.add(item_id)
            if len(tokens) == 4:
                user_item_ratings[(user_id, item_id)] = (rating, timestamp)
            else:
                user_item_ratings[(user_id, item_id)] = rating
            if user_id not in user_items_dict:
                user_items_dict[user_id] = []
                user_items_dict[user_id].append(item_id)
            else:
                user_items_dict[user_id].append(item_id)

            if item_id not in item_users_dict:
                item_users_dict[item_id] = []
                item_users_dict[item_id].append(user_id)
            else:
                item_users_dict[item_id].append(user_id)
    return list(users), list(items), user_items_dict, item_users_dict, user_item_ratings

def get_users_from_social_adj_file(filename):
    users = set()
    edges = []
    with open(filename) as fr:
        for line in fr.readlines():
            tokens = line.split()
            user_1 = int(tokens[0])
            user_2 = int(tokens[1])
            # For now we omit the weight
            # rating = float(tokens[2])

            users.add(user_1)
            users.add(user_2)

            edges.append((user_1, user_2))

    return list(users), edges

def filter_and_remap_dataset(user_items_dict_full, user_item_ratings, users_social, edges_social, num_ratings_threshold):
    filtered_users_social = set()
    filtered_users_social_gcc = set()
    filtered_edges_social = set()
    filtered_edges_social_gcc = set()
    print('Filtering edges...')

    G = nx.Graph()
    for (user_1, user_2) in tqdm(edges_social):
        if user_1 not in user_items_dict_full or user_2 not in user_items_dict_full:
            continue
        if len(user_items_dict_full[user_1]) > num_ratings_threshold and len(user_items_dict_full[user_2]) > num_ratings_threshold:
            filtered_users_social.add(user_1)
            filtered_users_social.add(user_2)
            filtered_edges_social.add((user_1, user_2))
            G.add_edge(user_1, user_2)

    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G0 = G.subgraph(Gcc[0])

    print('Filtering edges in giant connected component...')
    for (user_1, user_2) in tqdm(filtered_edges_social):
        if user_1 in G0.nodes() and user_2 in G0.nodes():
            filtered_users_social_gcc.add(user_1)
            filtered_users_social_gcc.add(user_2)
            filtered_edges_social_gcc.add((user_1, user_2))

    filtered_user_items_dict_full = {}
    filtered_item_users_dict_full = {}
    filtered_user_item_ratings = {}

    remapped_filtered_user_items_dict_full = {}
    remapped_filtered_item_users_dict_full = {}
    remapped_filtered_user_item_ratings = {}

    user_id_mappings = {}
    item_id_mappings = {}

    user_id_counter = 0
    item_id_counter = 0

    print('Filtering ratings...')
    for user_id in tqdm(user_items_dict_full):
        # User is part of the dataset if it has more than threshold ratings and has social connections
        if len(user_items_dict_full[user_id]) > num_ratings_threshold and (user_id in filtered_users_social_gcc):
            if user_id not in user_id_mappings:
                user_id_mappings[user_id] = user_id_counter
                user_id_counter += 1
            user_id_remapped = user_id_mappings[user_id]

            filtered_user_items_dict_full[user_id] = []
            remapped_filtered_user_items_dict_full[user_id_remapped] = []

            for item_id in user_items_dict_full[user_id]:
                if item_id not in item_id_mappings:
                    item_id_mappings[item_id] = item_id_counter
                    item_id_counter += 1
                item_id_remapped = item_id_mappings[item_id]

                if item_id not in filtered_item_users_dict_full:
                    filtered_item_users_dict_full[item_id] = []
                    remapped_filtered_item_users_dict_full[item_id_remapped] = []

                filtered_user_items_dict_full[user_id].append(item_id)
                filtered_item_users_dict_full[item_id].append(user_id)
                filtered_user_item_ratings[(user_id, item_id)] = user_item_ratings[(user_id, item_id)]

                remapped_filtered_user_items_dict_full[user_id_remapped].append(item_id_remapped)
                remapped_filtered_item_users_dict_full[item_id_remapped].append(user_id_remapped)
                remapped_filtered_user_item_ratings[(user_id_remapped, item_id_remapped)] = user_item_ratings[(user_id, item_id)]

    remapped_filtered_nodes_social = set()
    remapped_filtered_edges_social = set()

    print('Remapping filtered edges...')
    for (user_1, user_2) in tqdm(filtered_edges_social_gcc):
        if user_1 not in user_id_mappings or user_2 not in user_id_mappings:
            continue
        remapped_filtered_edges_social.add((user_id_mappings[user_1], user_id_mappings[user_2]))
        remapped_filtered_edges_social.add((user_id_mappings[user_2], user_id_mappings[user_1]))

        remapped_filtered_nodes_social.add(user_id_mappings[user_1])
        remapped_filtered_nodes_social.add(user_id_mappings[user_2])

    return filtered_user_items_dict_full, \
           filtered_item_users_dict_full, \
           filtered_user_item_ratings, \
           remapped_filtered_user_items_dict_full, \
           remapped_filtered_item_users_dict_full, \
           remapped_filtered_user_item_ratings, \
           remapped_filtered_nodes_social, \
           remapped_filtered_edges_social, \
           user_id_mappings, \
           item_id_mappings

In [20]:
num_ratings_threshold = 2
dataset_names =['toy_dataset', 'epinions', 'ciao']

for dataset_name in dataset_names:
    print(dataset_name)
    print('======================================')
    ratings_map = create_ratings_map('../data/' + dataset_name + '/ratings.tsv')
    # users_train, items_train, user_items_dict_train, item_users_dict_train, users_items_ratings_train = get_users_and_items_from_ratings_file('../data/' + dataset_name + '/train.tsv', ratings_map)
    # users_test, items_test, user_items_dict_test, item_users_dict_test, users_items_ratings_test = get_users_and_items_from_ratings_file('../data/' + dataset_name + '/test.tsv', ratings_map)
    users_full, items_full, user_items_dict_full, item_users_dict_full, users_items_ratings_full = get_users_and_items_from_ratings_file('../data/' + dataset_name + '/ratings.tsv', ratings_map)
    users_social, edges_social = get_users_from_social_adj_file('../data/' + dataset_name + '/social_connections.tsv')

    filtered_user_items_dict_full, \
    filtered_item_users_dict_full, \
    filtered_user_item_ratings, \
    remapped_filtered_user_items_dict_full, \
    remapped_filtered_item_users_dict_full, \
    remapped_filtered_user_item_ratings, \
    remapped_filtered_nodes_social, \
    remapped_filtered_edges_social, \
    user_id_mappings, \
    item_id_mappings = filter_and_remap_dataset(user_items_dict_full, users_items_ratings_full, users_social, edges_social, num_ratings_threshold)

    with open('../data/' + dataset_name + '/filtered_social_connections.tsv', 'w') as the_file:
       for social_edge in remapped_filtered_edges_social:
           the_file.write(f'{social_edge[0]}\t{social_edge[1]}\t1\n')

    with open('../data/' + dataset_name + '/filtered_ratings.tsv', 'w') as the_file:
       for (remapped_user_id, remapped_item_id) in remapped_filtered_user_item_ratings:
           value = remapped_filtered_user_item_ratings[(remapped_user_id, remapped_item_id)]
           if type(value) == tuple:
               the_file.write(f'{remapped_user_id}\t{remapped_item_id}\t{value[0]}\t{value[1]}\n')
           else:
               the_file.write(f'{remapped_user_id}\t{remapped_item_id}\t{value}\n')

toy_dataset
Filtering edges...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2252/2252 [00:00<00:00, 411249.24it/s]


Filtering edges in giant connected component...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2004/2004 [00:00<00:00, 539914.26it/s]


Filtering ratings...


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12883/12883 [00:00<00:00, 518587.08it/s]


Filtering ratings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17615/17615 [00:00<00:00, 237409.87it/s]


Remapping filtered edges...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12873/12873 [00:00<00:00, 961692.71it/s]


In [7]:
remapped_filtered_user_item_ratings[(0, 0)]

0.0

In [27]:
print(len(users_full))
print(len(items_full))
print(len(users_items_ratings_full))
print(len(users_social))
print(len(edges_social))
print('\n')
print(len(filtered_user_items_dict_full))
print(len(filtered_item_users_dict_full))
print(len(filtered_user_item_ratings))
print('\n')
print(len(remapped_filtered_user_items_dict_full))
print(len(remapped_filtered_item_users_dict_full))
print(len(remapped_filtered_user_item_ratings))
print(len(remapped_filtered_nodes_social))
print(len(remapped_filtered_edges_social))
print('\n')
print(len(user_id_mappings))
print(len(item_id_mappings))

40163
139738
664823
49287
487182


27202
137536
638067


27202
137536
638067
27202
799792


27202
137536


In [28]:
G = nx.Graph()
for edge in remapped_filtered_edges_social:
    G.add_edge(edge[0], edge[1])
nx.number_connected_components(G)
for component in nx.connected_components(G):
    print(len(component))

Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G0 = G.subgraph(Gcc[0])

27202


27202

In [12]:
G = nx.Graph()
for edge in edges_social:
    G.add_edge(edge[0], edge[1])
nx.number_connected_components(G)
for component in nx.connected_components(G):
    print(len(component))

49287


In [19]:
len(item_id_mappings)

137536

In [ ]:
G0